In [13]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [15]:
# Gather Data

boston_dataset = load_boston()
data = pd.DataFrame(data=boston_dataset.data, 
                    columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])
#features.head()

In [19]:
CRIM_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = features.mean().values.reshape(1, 11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [17]:
regr = LinearRegression().fit(features, target)
fitted_vals = regr.predict(features)

# Challenge: calculate the MSE and the RMSE using sklearn
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [21]:
def get_log_estimate(nr_rooms, students_per_classroom, 
                     next_to_river=False, high_confidence=True):
    # configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = students_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # calc range
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval    

In [22]:
get_log_estimate(3, 20, next_to_river=True)

(2.776758191480399, 3.1517824618746597, 2.4017339210861386, 95)

In [23]:
np.median(boston_dataset.target)

21.2

In [29]:
# Challenge: write the code that converts the log price estimate using the 1970s prices 
# as well as the upper and lower bounds to today's prices. Round the values to the 
# nearest 1000 dollars

ZILLOW_MEDIAN_PRICE = 583.6
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, students_per_classroom=15,
                                              next_to_river=False, high_confidence=False)

# Convert to today's dollars
dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
dollar_high = np.e**upper * 1000 * SCALE_FACTOR
dollar_low = np.e**lower * 1000 * SCALE_FACTOR

# Round the dollar values to nearest thousand
rounded_est = np.around(dollar_est, -3)
rounded_high = np.around(dollar_high, -3)
rounded_low = np.around(dollar_low, -3)

print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_low} at the lower end to USD {rounded_high} at the high end')

The estimated property value is 827000.0.
At 68% confidence the valuation range is
USD 686000.0 at the lower end to USD 998000.0 at the high end


In [42]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    
    """
    Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property
    ptratio -- number of students per teacher in the classrom for the schools in the area
    chas -- True if the property is next to the river, False otherwise
    large-range -- True for a 95% prediction interval, False for a 68% interval
    """
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again!')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, ptratio,
                                              next_to_river=chas, high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_high = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_high = np.around(dollar_high, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_high} at the high end')

In [41]:
get_dollar_estimate(rm=1, ptratio=50, chas=True)

The estimated property value is 120000.0.
At 95% confidence the valuation range is
USD 83000.0 at the lower end to USD 175000.0 at the high end


In [43]:
import boston_valuation as val
val.get_dollar_estimate(6, 12, True)

The estimated property value is 783000.0.
At 95% confidence the valuation range is
USD 538000.0 at the lower end to USD 1140000.0 at the high end
